In [ ]:
import pandas as pd
import time
from pathlib import Path
import sys
import os
import multiprocessing
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import requests
import glob
import pickle
import fitz
from string import digits
# from Codes.Section_01_Data_Extraction_Preparation.file_preparation import download_file, rotate_pdf, pickle_pdf_xml
# from Codes.Section_01_Data_Extraction_Preparation.pdf_metadata import get_pdf_metadata

In [ ]:
%cd "c:\\Users\\t1nipun\\Desktop\\BERDI\\esa-data-bank_banque-donnees-ees"

### 1. Download all PDFs

In [ ]:
### Read all the names of the PDFs to be downloaded
path = os.getcwd()
Index0_path = path + '\\data\\raw\\index_for_projects\\Phase3_Index_of_PDFs_for_Major_Projects_with_ESAs.csv'

Index0 = pd.read_csv(Index0_path, encoding= 'unicode_escape')
print("Number of PDF Files to be downloaded: ", len(Index0))

Index0 = Index0.rename(columns={'Application Short Name': 'App_Short_Name'})

Index0.head()

In [ ]:
### Function to download the PDFs

def download_file(path, Index0):
    count = 0
    error_urls = []
    error_dataIDs = []
    dataID = ""

    # Iterating each row in the Index0 dataframe
    for index, row in Index0.iterrows():
        try:
            dataID = row['DataID']
            download_url = 'http://docs2.cer-rec.gc.ca/ll-eng/llisapi.dll?func=ll&objId=' + str(dataID) + '&objaction=download&viewType=1'
            r = requests.get(download_url)  # scraping the PDF file from the URL
            full_name = os.path.join(path + 'Data_Files_v2\\PDFs\\' + str(dataID) + '.pdf')
            
            with open(full_name, 'wb') as file:
                file.write(r.content)
            count = count + 1
            
            #print(dataID)
        except:
            # storing the error logs
            error_urls.append(download_url)
            error_dataIDs.append(dataID)
            print("error with file {}".format(row['File Name']))

    # creating and and saving the error logs dataframe
    df_scraping_errorlog = pd.DataFrame({'error_dataIDs': error_dataIDs,
                                         'error_urls': error_urls
                                         })
    df_scraping_errorlog.to_csv(path + 'Data_Files_v2\\Error_Logs\\ScrapingPDFErrorLogs.csv', index=False, encoding='utf-8-sig')
    
    print("Count of the PDFs to be downloaded: ", len(Index0))
    print("Files successfully downloaded: ", count)
    print("Count of Files with Errors: ", len(df_scraping_errorlog))
    return count


In [ ]:
# Download files
count = download_file(path, Index0)
print("Files downloaded ", count)

In [ ]:
pdf_dataID = []
for index, row in Index0.iterrows():
    pdf_dataID.append(row['Data ID'])
    
print(len(pdf_dataID))
print(pdf_dataID[0])

### 2. Extract Features for each page of the PDFs downloaded 

In [ ]:
def count_word_in_str(long_str, key):
    count = 0
    for word in long_str.split():
        if key in word:
            count = count + 1
    return(count)
#count_word_in_str("vibudh rocks dh dh ddh","dh")

In [ ]:
def count_words_in_str(long_str, keys):
    count = 0
    for word in long_str.split():
        for key in keys:
            if key in word:
                count = count + 1
    return(count)
#count_words_in_str("vibudh rocks dh dh ddh",["dh","vi"])

In [ ]:
def area_of_imgblocks(imgblocks):
    sum_areas = 0
    for imgblock in imgblocks:
        block_area = imgblock['width']* imgblock['height']
        sum_areas = sum_areas + block_area
    return(sum_areas)

In [ ]:
s = 'abc123def456ghi789ersuit834678 dhfuaiwhbui34tr234 zero0'
remove_digits = str.maketrans('', '', digits)
res = s.translate(remove_digits)
res

In [ ]:
def extract_features(dataIDs):
    words_in_page = []
    
    scale = []
    km_kilometers = []
    m = []
    metres = []
    scale_grp = []
    
    legend = []
    
    figure = []
    mapp = []
    alignment_sheet = []
    sheet = []
    figure_grp = []
    
    north = []
    n = []
    north_grp = []
    
    dataID_pageNo = []
    
    count = 0
    
    No_of_images = []
    Area_of_images = []
    cnt = 0
    
    dataID_l = []
    page_no = []
    paths_l = []
    
    error_files = []
    i = 0
    
    for dataID in dataIDs:
        pdf_path = path + '\\data\\raw\\pdfs\\' + str(dataID) + '.pdf'
        print(pdf_path)
        i = i+1 
        print("File Starting: {}. PDF {} out of {}".format(dataID, i, len(dataIDs)))     
        
        try:
        
            j = 0    
            doc = fitz.open(pdf_path)             
            #print("hey", str(len(doc)))
            for page in doc:  # iterate through the pages
                print(page)
                j = j+1 #Number of pages
                print(j)
                cnt = cnt + 1
                print(cnt)
                p = page.get_text("dict")
            
                blocks = p["blocks"]
                imgblocks = [b for b in blocks if b["type"] == 1]
                No_of_images.append(len(imgblocks))
                Area_of_images.append(area_of_imgblocks(imgblocks))
                
            
                p = str(p).replace('<p>', '').replace('</p>','').replace(".",'').replace(",",'').replace('"','').lower()
                p = p.translate(remove_digits)
                
                words_lst = p.split()
                
                word_count = 0
                big_words = ""
                words = ""
                #print(pdf_path)
                for word in words_lst:
                    words  = words + " " + word
                    if len(word) > 3:
                        word_count = word_count + 1
                        big_words = big_words + " " + word
                        
                words_in_page.append(word_count)
                #print(pdf_path)
                
                sc_grp = 0 
                if "scale" in big_words:
                    scale.append(count_word_in_str(big_words, "scale"))
                    sc_grp = 1
                else:
                    scale.append(0)
                    
                if ("kilometre" in big_words or "kilometer" in big_words or "km " in p):
                    km_kilometers.append(count_words_in_str(p, ["kilometre", "kilometer", "km "]))
                    sc_grp = 1
                else:
                    km_kilometers.append(0)
                    
                if("m " in p):
                    m.append(count_word_in_str(p, "m "))
                else:
                    m.append(0)
                    
                if("metre" in big_words or "meter" in big_words):
                    metres.append(count_words_in_str(big_words, ["meter","metre"]))
                    sc_grp = 1
                else:
                    metres.append(0)
                    
                if sc_grp > 0:
                    scale_grp.append(1)
                else:
                    scale_grp.append(0)
                    
                
                
                if "legend" in big_words:
                    legend.append(count_word_in_str(big_words,"legend"))
                else:
                    legend.append(0)
                
                    
                    
                fig_grp = 0
                if "figure" in big_words:
                    figure.append(count_word_in_str(big_words,"figure"))
                    fig_grp = 1
                else:
                    figure.append(0)
                    
                if "map " in p:
                    mapp.append(count_word_in_str(p,"map "))
                    fig_grp = 1
                else:
                    mapp.append(0)
                      
                if "alignment sheet" in big_words:
                    alignment_sheet.append(1)
                    fig_grp = 1
                else:
                    alignment_sheet.append(0)
                    
                if "sheet" in big_words:
                    sheet.append(count_word_in_str(big_words,"sheet"))
                    fig_grp = 1
                else:
                    sheet.append(0)
                
                if fig_grp > 0:
                    figure_grp.append(1)
                else:
                    figure_grp.append(0)
                
                
                   
                if "north" in big_words:
                    north.append(count_word_in_str(big_words, "north"))
                    no_grp = 1
                else:
                    north.append(0)
                    
                if "n" in p:
                    n.append(count_word_in_str(p, " n "))
                    no_grp = 1
                else:
                    n.append(0)
                          
                dataID_l.append(dataID)
                page_no.append(j)
                paths_l.append(pdf_path)
                
        
        except:
            #if 1==0:
            print("Error Found")
            error_files.append(dataID)
            page_no.append(j)

        
    Features = pd.DataFrame({'scale' : scale, 
                           'km_kilometers' : km_kilometers, 
                           'm' : m, 
                           'metres' : metres, 
                           'scale_grp' : scale_grp, 
                           'legend' : legend, 
                           'figure' : figure, 
                           'mapp' : mapp, 
                           'alignment_sheet' : alignment_sheet, 
                           'sheet' : sheet, 
                           'figure_grp' : figure_grp, 
                           'north' : north, 
                           'n' : n, 
                           'words_in_page' : words_in_page,
                            'No_of_images' : No_of_images, 
                            'Area_of_images' : Area_of_images,
                           'dataID' : dataID_l, 
                            'pageNo': page_no, 
                             'paths_l': paths_l
                           #'Y_class' : Y_class
                           })
    pages_ref = pd.DataFrame({'DataIDs': dataID_l,
                            'pdf_paths' : paths_l, 
                            'Page_no': page_no})
     
       
    #print("Total Number of pages processed: {}".format(count))   
    return Features, pages_ref, error_files

In [ ]:
#"hello " + 3
X_df, pages_ref, error_files = extract_features(pdf_dataID) 
#Features
#dataIDs
#error_files



In [ ]:
print("Total PDFs", len(pdf_dataID))
print("Total No. of Pages", len(X_df))
print("Total Error Files", len(error_files))

In [ ]:
print(len(X_df))
pd.set_option("display.max_rows", None)
X_df

In [48]:
X_df.to_csv(path + '\\data\\processed\\page_features\\all_features.csv')